## Get Transcripts from YouTube Videos

In [ ]:
#ensure you've install packages from requirements.txt

In [ ]:
# Download the youtube video transcript
from youtube_transcript_api import YouTubeTranscriptApi
import re

# Replace 'video_id' with the ID of the YouTube video for your fav Keynote address (e.g. Ignite, re:Invent, Next..)
# Find the video you want and strip off the portion of the url following the '/watch?v='

#video_id = 'FZhbJZEgKQ4' #MS-2023
#video_id = 'iuU2YJohuZE' #IntelInnovation2023
#video_id = '3zR5tx_Jm6k' #MSIgnite2017
#video_id = 'sgW_ww1Xxe8&t=631s' #MS-2019
#video_id = 'lEUS3_Qjyjg&t=2757s' #GCP_NEXT_2023
#video_id = '8clH7cbnIQw' #AWS-2023
#video_id = '1IxDLeFQKPk' #AWS-2017-Andy
#video_id = 'nFKVzEAm-ts' #AWS-2017-Vogel
video_id = 'pJG6nmR7XxI&t=16s' #AWS-2017-Pete
#video_id = '-bxTA8vqoIU' #ServiceNow2023

# Get the transcript from the youtube
transcript_list = YouTubeTranscriptApi.get_transcript(video_id)

# Join the :text: portion up
transcript_text = ' '.join([transcript['text'] for transcript in transcript_list])

# Fix some transcript goofs for tech words jargon
transcript_text = re.sub(r'\bsequel\b', 'SQL', transcript_text, flags=re.IGNORECASE) # replace 'sequel' with 'SQL', doh.
## add more here, like IoT or YAML or other things that yt transcript fouls up


## Remove Stopwords, WordCloud Keyword Frequency

In [ ]:
# Remove STOP WORDS like "at", "the" and "and" - then create a word cloud by frequency of key terms in the transcript
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# Download dics of words
nltk.download('stopwords')
nltk.download('punkt')

# Tokenize
words = word_tokenize(transcript_text)

# Filter out stop words and junk words
stop_words = set(stopwords.words('english'))
more_stop_words = ['use', 'well', 'azure', 'new', 'one', 'know', 'see', 'actually', 'let', 'sort', 'yeah', 'able', 'need', 'going', 'show', 'take', \
                   'microsoft', 'thomas', 'aws', 'help', 'gon', 'na', 'us', 'uh', 'way', 'right', 'much', 'really', 'want', 'get', 'google', 'amazon', 'also', 'like' \
                   'thank', 'like', 'thank']
all_stop_words = stop_words.union(more_stop_words)
keywords = [word.lower() for word in words if word.isalpha() and word.lower() not in all_stop_words]

# Calculate freq. dist. of the words remaining 'keywords'
freq_dist = nltk.FreqDist(keywords)

# Get the top N most common, after remove basic words, replace (N) w/ the TopN you want.
common_words = freq_dist.most_common(10)

# Create a string
word_freq = {word: freq for word, freq in common_words}

# WordCloud (WC)
wordcloud = WordCloud(width = 800, height = 800, 
                      background_color ='white', 
                      min_font_size = 10)

# Generage the WC
wordcloud.generate_from_frequencies(word_freq)

# Plot and show the WC           
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud)
plt.tight_layout(pad = 0)
plt.axis("off") 
plt.show()